In [ ]:
import numpy as np
import scipp as sc

# Tips, tricks, and anti-patterns
## Choose dimensions wisely

A good choice of dimensions for representing data goes a long way in making working with Scipp efficient.
Consider, e.g., data gathered from detector pixels at certain time intervals.
We could represent it as

In [ ]:
npix = 100
ntime = 10
data = sc.zeros(dims=['pixel','time'], shape=[npix, ntime])
data

For irregularly spaced detectors this may well be the correct or only choice.
If however the pixels are actually forming a regular 2-D image sensor we should probably prefer

In [ ]:
nx = 10
ny = npix // nx
data = sc.zeros(dims=['y', 'x', 'time'], shape=[ny, nx, ntime])
data

With this layout we can naturally perform slices, access neighboring pixel rows or columns, or sum over rows or columns.

## Choose dimension order wisely

In principle the order of dimensions in Scipp can be arbitrary since operations transpose automatically based on dimension labels.
In practice however a bad choice of dimension order can lead to performance bottlenecks.
This is most obvious when slicing multi-dimensional variables or arrays, where slicing any but the outer dimension yields a slice with gaps between data values, i.e., a very inefficient memory layout.
If an application requires slicing (directly or indirectly, e.g., in `groupby` operations) predominantly for a certain dimension, this dimension should be made the *outermost* dimension.
For example, for a stack of images the best choice would typically be

In [ ]:
nimage = 13
images = sc.zeros(dims=['image', 'y', 'x'], shape=[nimage, ny, nx,])
images

Slices such as

In [ ]:
images['image', 3]

will then have data for all pixels in a contiguous chunk of memory.
Note that in Scipp the first listed dimension in `dims` is always the *outermost* dimension (NumPy's default).

## Avoid loops

With Scipp, just like with NumPy or Matlab, loops such as `for`-loops should be avoided.
Loops typically lead to many small slices or many small array objects and rapidly lead to very inefficient code.
If we encounter the need for a loop in a workflow using Scipp we should try and take a step back to understand how it can be avoided.
Some tips to do this include:

### Use slicing with "shifts"

When access to neighbor slices is required, replace

In [ ]:
for i in range(len(images.values)-1):
    images['image', i] -= images['image', i+1]

with

In [ ]:
images['image', :-1] -= images['image', 1:]

Note that at this point NumPy provides more powerful functions such as [numpy.roll](https://numpy.org/doc/stable/reference/generated/numpy.roll.html).
Scipp's toolset for such purposes is not fully developed yet.

### Seek advice from NumPy

There is a huge amount of information available for NumPy, e.g., on [stackoverflow](https://stackoverflow.com/questions/tagged/numpy?tab=Votes).
We can profit in two ways from this.
In some cases, the same techniques can be applied to Scipp variables or data arrays, since mechanisms such as slicing and basic operations are very similar.
In other cases, e.g., when functionality is not available in Scipp yet, we can resort to processing the raw array accessible through the `values` property:

In [ ]:
var = sc.arange('x', 10.0)
var.values = np.roll(var.values, 2)
var

The `values` property can also be used as the `out` argument that many NumPy functions support:

In [ ]:
np.exp(var.values, out=var.values)
var

<div class="alert alert-warning">
    <b>WARNING</b>

When applying NumPy functions to the `values` directly we lose handling of units and variances, so this should be used with care.
</div>

### Use helper dimensions or reshaped data

Some operations may be difficult to implement without a loop in a certain data layout.
If this layout cannot be changed globally, we can still change it temporarily for a certain operation.
Even if this requires a copy it may still be faster and more concise than implementing the operation with a loop.
For example, we can sum neighboring elements by temporarily reshaping with a helper dimension using `fold`:

In [ ]:
var = sc.arange('x', 12.0)
var.fold('x', sizes={'x': 4, 'neighbors': 3}).sum('neighbors')

Note that `fold` returns a view, i.e., the operation is performed without making a copy of the underlying data buffers.
The companion operation of `fold` is `flatten`, which provides the reverse operation (see the [section below](#reshaping-data) for more details).

## Use in-place operations

Allocating memory or copying data is an expensive process and may even be the dominant factor for overall application performance, apart from loading large amounts of data from disk.
Therefore, it pays off the avoid copies where possible.

Scipp provides two mechanisms for this, in-place arithmetic operators such as `+=`, and `out`-arguments similar to what NumPy provides.
Examples:

In [ ]:
var = var * 2.0 # makes a copy
var *= 2.0 # in-place (faster)

In [ ]:
var = sc.sqrt(var) # makes a copy
var = sc.sqrt(var, out=var) # in-place (faster)

Note that in-place operations cannot be used if a broadcast is required or a dtype change happens, since in-place operations may only change the data contained in a variable.

## Reshaping data

<div id='reshaping-data'></div>

The shape of a `Variable` or a `DataArray` can be modified using the [fold](https://scipp.github.io/generated/functions/scipp.fold.html) and
[flatten](https://scipp.github.io/generated/functions/scipp.flatten.html) functions.
Below are a few examples to illustrate how they work.

### Folding

In a nutshell, the `fold` operation increases the number of dimensions of the data.
We begin with a two-dimensional variable:

In [ ]:
N = 4
M = 3
var = sc.array(dims=['x', 'y'], values=np.random.random([N,M]))
sc.show(var)

We then fold the `x` dimension into two new dimensions `a` and `b`:

In [ ]:
folded_var = sc.fold(var, dim='x', sizes={'a': 2, 'b': 2})
sc.show(folded_var)

The result is a three-dimensional variable with dimensions `(a, b, y)`.

A `DataArray` with coordinates can also be folded:

In [ ]:
x = sc.array(dims=['x'], values=np.arange(N))
y = sc.array(dims=['y'], values=np.arange(M))
da = sc.DataArray(data=var, coords={'x':x,'y':y})
sc.show(da)

In [ ]:
folded_da = sc.fold(da, dim='x', sizes={'a': 2, 'b': 2})
sc.show(folded_da)

Note that the dimensions of the `x` coordinate have changed from `(x,)` to `(a, b)`,
but the coordinate name has not changed.

### Flattening

The inverse of the `fold` operation is `flatten`.
This is analogous to NumPy's [flatten](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html) method.
By default, all dimensions of the input are flattened to a single dimension,
whose name is provided by the `to` argument:

In [ ]:
flat_da = sc.flatten(da, to='z')
sc.show(flat_da)

It is however possible to only flatten selected dimensions, using the `dims` argument.
For example, we can flatten the `a` and `b` dimensions of our previously folded (three-dimensional) data to recover a two-dimensional array.

In [ ]:
flat_ab = sc.flatten(folded_da, dims=['a', 'b'], to='time')
sc.show(flat_ab)

### Stacking: concatenating into a new dimension

Another very common operation is stacking together arrays that have the same shape.
In NumPy, this is called [stack](https://numpy.org/doc/stable/reference/generated/numpy.stack.html).
In Scipp, this is achieved using [concat](https://scipp.github.io/generated/functions/scipp.concat.html).

The most widely known use of `concat` is probably to concatenate data along an existing dimension.
With Scipp's labeled dimensions, it is however also possible to concatenate variables and data arrays along a new dimension,
which is equivalent to NumPy's `stack`.

In [ ]:
stacked = sc.concat([da, da], dim='z')
sc.show(stacked)